<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'>Editado por Equipo Docente IIC2233 2018-1 y 2018-2, basado en documento de Nebil Kawas 2017-2.</font><br>
<font size='1'>Incluye partes de un material confeccionado por Karim Pichara y Christian Pieringer en 2015.</font>
<br>
</p>

## Decoradores de funciones

En diseño de _software_, un decorador es un patrón de diseño en el que se añade una funcionalidad a un objeto, sin tener que reescribir el código original. Los decoradores de funciones aplican esa idea: permiten tomar una función ya implementada, agregar algún comportamiento o datos adicionales, y retornar una nueva función. 

Podemos ver los decoradores como funciones que reciben una función `f1` cualquiera, y retornan una función `f2` distinta. Por ejemplo, si nuestro decorador se llama `decorator`, para obtener la función modificada que queremos y asignarla a la misma función actual, simplemente escribimos `f1 = decorator(f1)`. Con esto, nuestra función `f1` ahora queda con los nuevos datos y comportamientos agregados. 

Un beneficio de los decoradores es que evitan la necesidad de modificar el código de la función original, por lo que si necesitamos volver a la versión inicial de la función simplemente quitamos el llamado al decorador.

Empecemos por algo sencillo: definamos el decorador identidad. Y le decimos _identidad_ porque, simplemente, devuelve una función (el _wrapper_) que, una vez llamado, ejecutará la función original.

In [1]:
def deco_function(original_function):
    print("Entrando... (1)")

    def wrapper_function():
        print("Entrando... (2)")
        original_function()
        print("Saliendo... (2)")

    print("Saliendo... (1)")
    return wrapper_function

Pensemos que estamos en nuestra época favorita del año, como pueden ser las fiestas patrias. Ahora, definimos una función dieciochera.

In [2]:
def print_paya():
    print("¡Aro, aro, aro!")

Llamamos a esta función recién definida.

In [3]:
print_paya()

¡Aro, aro, aro!


Le entregamos la función al decorador —recordemos que el decorador también es una función— para crear una **nueva** función.

In [4]:
new_print_paya = deco_function(print_paya)

Entrando... (1)
Saliendo... (1)


La función decorada debería tener el nuevo comportamiento.

In [5]:
new_print_paya()

Entrando... (2)
¡Aro, aro, aro!
Saliendo... (2)


Y efectivamente lo tiene.  
Pero también podemos utilizar **el nombre de la función original** como variable.

In [6]:
print_paya = deco_function(print_paya)

Entrando... (1)
Saliendo... (1)


Y obtendremos el mismo resultado.

In [7]:
print_paya()

Entrando... (2)
¡Aro, aro, aro!
Saliendo... (2)


### Decoradores con *azúcar sintáctico*  

Una forma equivalente, pero más rápida y legible de decorar funciones es escribiendo el nombre del decorador arriba del encabezado de la función anteponiendo un `@`. Es la misma sintáxis que usamos cuando queremos crear properties, de hecho, `property` es un decorador.

Nuestra paya decorada de esta manera, quedaría:

In [8]:
@deco_function
def print_paya():
    print("¡Aro, aro, aro!")

Entrando... (1)
Saliendo... (1)


Podemos ver que la función `print_paya` tendrá el nuevo comportamiento cuando la llamemos.

In [9]:
print_paya()

Entrando... (2)
¡Aro, aro, aro!
Saliendo... (2)


Veamos un ejemplo, ahora, con una nueva función vegetariana.

In [10]:
@deco_function
def print_comida():
    print("Comí pimentones con huevo.")
    print("Comí empanadas vegetarianas.")

Entrando... (1)
Saliendo... (1)


Veamos el resultado.

In [11]:
print_comida()

Entrando... (2)
Comí pimentones con huevo.
Comí empanadas vegetarianas.
Saliendo... (2)


El decorador se aplicó satisfactoriamente.

### _Intermezzo_

Hemos visto cómo aplicar un decorador de dos formas equivalentes, pero los hemos aplicado sólo sobre funciones sin parámetros. Intentemos con una nueva función que, a diferencia de las anteriores, acepta un parámetro.

In [12]:
@deco_function
def print_bebida(bebida):
    print(f"Este dieciocho, me tomé dos litros de {bebida} al día.")

Entrando... (1)
Saliendo... (1)


Veamos nuevamente el resultado.

In [13]:
print_bebida("agua")

TypeError: wrapper_function() takes 0 positional arguments but 1 was given

Algo salió mal. El _wrapper_ no esperaba recibir un argumento. 

Podríamos resolver este problema colocando un parámetro a `wrapper_function` que está definido en nuestro decorador. Sin embargo, con esa solución sólo podremos decorar funciones que reciben exactamente un parámetro. Lo que a nosotros nos gustaría es poder decorar cualquier función, independiente de la cantidad de parámetros que reciba.

Para resolver este problema en forma definitiva, necesitamos utilizar `*args` y `**kwargs`.

- “¿Qué es `*args` y `**kwargs`?”  
    - Ellos son utilizados generalmente en la definición de funciones, y sirven para pasar una **cantidad variable** de argumentos.  

- “¿Y para qué me sirve eso?”  
    - Esto me será de gran utilidad en casos cuando yo no sepa, de antemano, cuántos argumentos me llegarán.

Veamos un ejemplo simple.

In [14]:
def multiply_two_numbers(first, second):
    return first * second

Utilicemos esta función para multiplicar dos números.

In [15]:
multiply_two_numbers(6, 7)

42

¿Y cómo hago si quiero multiplicar una cantidad variable de números?

In [16]:
from functools import reduce

def multiply_some_numbers(*numbers):
    print(numbers)  # Esto imprime el iterable completo.
    print(*numbers) # Esto imprime los números desempaquetados.
    return reduce(lambda x, y: x*y, numbers)

(Como puede inferir, no es necesario que tomen el nombre de `args` y `kwargs`.)  
Veamos cómo utilizarlo.

In [17]:
multiply_some_numbers(3, 4, 5, 7)

(3, 4, 5, 7)
3 4 5 7


420

#### Secreto

Y ahora, un secreto: la función de `print` utiliza esto por detrás.

In [18]:
print(42)
print("Esto", "permite", "imprimir", "un", "número", "variable", "de", "argumentos.")

42
Esto permite imprimir un número variable de argumentos.


### Decoradores para funciones con cualquier cantidad de argumentos

Entonces, volviendo al primer decorador…  
Ahora, agreguemos los `*args` y `**kwargs`.

In [19]:
def deco_function(original_function):
    print("Entrando... (1)")
    
    def wrapper_function(*args, **kwargs):    
        print("Entrando... (2)")
        original_function(*args, **kwargs)
        print("Saliendo... (2)")
    
    print("Saliendo... (1)")
    return wrapper_function

In [20]:
@deco_function
def print_bebida(bebida):
    print(f"Este dieciocho, me tomé dos litros de {bebida} por día.")

Entrando... (1)
Saliendo... (1)


Veamos nuevamente el resultado.

In [21]:
print_bebida("pipeño")

Entrando... (2)
Este dieciocho, me tomé dos litros de pipeño por día.
Saliendo... (2)


Ahora sí funcionó. 😀

### Decoradores con parámetros

Si queremos crear decoradores que acepten parámetros, debemos agregar un tercer nivel de funciones anidadas. Cada nivel tiene un rol:

- La función más externa es el constructor del decorador.
- La función intermedia es el decorador.
- La función más interna es la función modificada.

En general, esta estructura se ve de la siguiente manera:

In [22]:
# Función creadora de decoradores. 
# Recibe parámetros para personalizar nuestro decorador.
def my_decorator_constructor(dec_parameters):
    # Función decoradora. Recibe sólo una función.
    def my_decorator(function): 
        # Wrapper. Acá podemos leer los argumentos recibidos.
        # en las dos funciones que están en niveles superiores.
        def wrapped_function(*args, **kwargs):
            # Hacer algo aquí antes de aplicar la función.
            # Llamar a la función y obtener lo que retorna.
            res = function(*args, **kwargs)
            # Hacer algo después.
            # Retornar un valor.
            return res
        return wrapped_function # Retorna la sub función.
    return my_decorator # Retorna el decorador.

**Veamos un ejemplo.** Siguiendo con nuestros ejemplos dieciocheros, primero definamos un decorador etílico sin parámetros:

In [23]:
# Sleep pausará la ejecución del programa por la cantidad de segundos que se indique
from time import sleep as caña

def caña_de_pipeño(original_function):
    def wrapper(*args, **kwargs):
        caña(3)  # Pausa por 3 segundos
        return original_function(*args, **kwargs)

    return wrapper

Definamos una simple función decorada.

In [24]:
@caña_de_pipeño
def add_twelve(number):
    return number + 12

Veamos el resultado.

In [25]:
print(f"¡Feliz {add_twelve(6)} para todos!")

¡Feliz 18 para todos!


La función demoró en responder.

Ahora, imaginemos que buscamos implementar lo mismo, pero con un parámetro que indique qué bebida tomamos.

In [26]:
def caña_de(bebida):
    def deco_function(original_function):
        def wrapper(*args, **kwargs):
            if bebida == "vino":
                caña(2)
                print("Ayuda, por favor.")
            elif bebida == "pipeño":
                caña(3)
                print("¿Dónde estoy? ¿Quién soy?")
                args = (0, )  # Para acrecentar los efectos del pipeño,
                              # podemos también cambiar los parámetros.
            else:
                print("No hay caña.")
            return original_function(*args, **kwargs)
        
        return wrapper
    return deco_function

Definimos la misma función, pero ahora con el decorador recién definido.

In [27]:
@caña_de("pipeño")
def add_twelve(number):
    return number + 12

Analicemos los efectos del pipeño.

In [28]:
print(f"¡Feliz {add_twelve(6)} para todos!")

¿Dónde estoy? ¿Quién soy?
¡Feliz 12 para todos!


Claro… por lo que ocurrió el [12 de febrero de 1818](https://es.wikipedia.org/wiki/Acta_de_Independencia_de_Chile).

Podemos ver que además de demorar 3 segundos en dar la respuesta, los argumentos dados a `add_twelve` fueron reemplazados con una tupla con un cero. De hecho, si intentamos con otro valor:

In [29]:
print(f"¡Feliz {add_twelve(33)} para todos!")

¿Dónde estoy? ¿Quién soy?
¡Feliz 12 para todos!


Obtenemos el mismo resultado.